Pablo Jimeno - UPV/EHU 2016

In [ ]:
# UNTESTED

%run 'Utilities_main.ipynb'

In [ ]:
## Load redMaPPer catalog:

redm_filedir = cat_dir + '/redMaPPer_DR12_v6.3.fits'
raw_redm = import_redMaPPer(redm_filedir)

# Compute weighted spectroscopic redshift
raw_redm['w_spec_z'] = weighted_redshift(raw_redm)

# Reduce centers to most probable CG position
raw_redm['ra']  = raw_redm['ra'][:,0]
raw_redm['dec'] = raw_redm['dec'][:,0]

# Apply redshift or richness cuts
#redm = create_subsample(redm, subkey='w_spec_z', min_val=0., max_val=0.9999)
#redm = create_subsample(redm, subkey='rich', min_val=20., max_val=999.)

In [ ]:
## Export catalogue
if 0:
    catalog_out = np.array((raw_redm['id'], 
                            raw_redm['ra'],
                            raw_redm['dec'],
                            raw_redm['w_spec_z'],
                            raw_redm['photo_z'],
                            raw_redm['photo_z_err'],
                            raw_redm['rich'],
                            raw_redm['rich_err'])).T

    # HEADER: ID / RA / DEC / W_SPEC_Z / PHOTO_Z / PHOTO_Z_ERR / RICHNESS / RICHNESS_ERR
    np.savetxt(cat_dir + '/redMaPPer_DR12_v6.3.txt', catalog_out,
               fmt='%.i\t%.7f\t%.7f\t%.6f\t%.6f\t%.6f\t%.2f\t  %.2f')

In [ ]:
## Richness mask:
rich_cutoff = 100.

## Redshift mask:
z_min = 0.00
z_max = 0.99


rich_mask = raw_redm['rich'] >= rich_cutoff
photo_z_mask = (raw_redm['photo_z'] >= z_min) & (raw_redm['photo_z'] <= z_max)
spec_z_mask = (raw_redm['w_spec_z'] == -1.) | (raw_redm['w_spec_z'] >= z_min)
redshift_z_mask = photo_z_mask & spec_z_mask

## Global mask:
global_mask = rich_mask & redshift_z_mask

print '\nNumber of redMaPPer clusters inside redshift and richness range: %i' % len(raw_redm['id'][global_mask])

## Construct sample:
redm = {}
mask = global_mask

for key in raw_redm.keys():
    redm[key] = raw_redm[key][mask]

In [ ]:
## Config:
"""http://wise2.ipac.caltech.edu/docs/release/allwise/expsup/index.html"""

"""WISE (...) mapped the sky at 3.4, 4.6, 12, and 22 μm (W1, W2, W3, W4) 
in 2010 with an angular resolution of 6.1", 6.4", 6.5", & 12.0" in the four bands."""

out_dir = '../Images/WISE/WISE_analysis'

img_pix = 600
img_mpc = 2.
#img_arcsec = 4.*60.

wise_band = 1

#min_arcsec = 0.
#max_arcsec = 120.
#nbins = 30
#r_bins = radial_binning(min_arcsec, max_arcsec, nbins)

In [ ]:
def generate_new_images(delete_fits=False):
    
    temp_dir = '%s/Temp_files' % data_dir
    if not os.path.isdir(temp_dir):
        os.makedirs(temp_dir)
    
    fits_dir = '%s/FITS' % raw_dir
    if os.path.exists(fits_dir):
        if delete_fits:
            shutil.rmtree(fits_dir)
            os.makedirs(fits_dir)
    else:
        os.makedirs(fits_dir)
        
    images_dirs = ['%s/PNG' % raw_dir, '%s/Lights' % img_raw_dir]
    for raw_dir in images_dirs:
        if os.path.exists(raw_dir):
            shutil.rmtree(raw_dir)
        os.makedirs(raw_dir)
        
        
generate_new_images(delete_fits=False)
        

from astropy.visualization import MinMaxInterval, PercentileInterval, AsymmetricPercentileInterval, SqrtStretch


## Run code:



for i, clu_id in enumerate(redm['id'][30:35]):

    clu = Cluster(redm, clu_id)

    img_scale, img_arcsec, img_arcmin, img_mpc = scale_image('mpc', clu.photo_z, img_pix, img_mpc)

    print 'Processing cluster nº %i (%06d)' % (i+1, clu_id)
    
    download_wise_fits(clu_id, clu.ra, clu.dec, img_arcsec, wise_band)
    wise_fits_to_png(clu_id, img_arcsec, img_pix, wise_band)  
    
    #fig = plot_spectrum_data(image_data)
    #fig.savefig('%s/check_%06d_power_v0.png' % (img_dir, clu_id))  
    image_data = get_wise_image_data(clu_id, img_arcsec, wise_band)
    image_data, img_mpc_size = zoom_image_data(image_data, img_pix, img_mpc)
    image_data = norm_image_absolute(image_data, norm_down=1., norm_up=300.)
        
    fig = plot_image_data(image_data)
    fig.savefig('%s/check_%06d_image_v1.png' % (img_dir, clu_id))    
    
    #fig = plot_histogram(image_data)
    #fig.savefig('%s/check_%06d_image_histo.png' % (img_dir, clu_id))    
    
    #fig = plot_spectrum_data(image_data)
    #fig.savefig('%s/check_%06d_power_v1.png' % (img_dir, clu_id))  
    
    plt.close()

In [ ]:
if 0:
    fluxes = []
    lights = []        

    for band in wise_bands:


        flux = compute_flux_distri(image_data, img_arcsec, r_bins)
        light = compute_light_distri(image_data, img_arcsec, r_bins)
        fluxes.append(flux)
        lights.append(light)

    cent_fit, back_fit, chi2 = fit_flux_distri(fluxes[0], r_bins)
    fit_light, half_light = check_light_distri(lights[0], cent_fit, back_fit, r_bins)
    half_lights.append(half_light)
    chi2s.append(chi2)
    #half_light_dist = process_half_light(img_z_photo, half_light)
    plot_flux_distribution(out_dir, img_id, wise_bands, fluxes, cent_fit, back_fit, r_bins)
    plot_flux_distribution_v2(out_dir, img_id, wise_bands, fluxes, cent_fit, back_fit, r_bins)
    #plot_light_distribution(out_dir, img_id, wise_bands, lights, cent_fit, back_fit, fit_light, half_light, r_bins)

    #visual_analysis(out_dir, img_id, band, img_rich, img_rich_err, img_z_photo,
    #                cent_fit, back_fit, chi2, half_light, half_light_dist)


In [ ]:
## Stacking:

img_concat = []

count = 0

for clu in range(5):

    count += 1
    img_id = clusters_id[clu]
    img_rich = clusters_rich[clu]
    img_rich_err = clusters_rich_err[clu]
    img_ra = clusters_ra[clu]
    img_dec = clusters_dec[clu]
    img_z_photo = clusters_z_photo[clu]
    img_z_spec = clusters_z_spec[clu]

    img_scale, img_arcsec, img_arcmin, img_mpc = scale_image('arcsec', img_z_photo, img_pix, img_arcsec)

    print 'Processing cluster nº %i (%06d)' % (count, img_id)
    
    download_wise_fits(img_id, img_ra, img_dec, img_arcsec, wise_bands)
    
    for band in wise_bands:
        
        img_data = get_data_wise_fits(img_id, img_arcsec, band)
        img_data_zoom = zoom_image_data(img_data, img_pix)
        img_concat.append(img_data_zoom)
                    
            
final_image = np.zeros(shape=img_concat[0].shape)

for image in img_concat:
    final_image += image

final_image = final_image/count

plot_image_data(final_image)
            
## Stacking:
#image_concat = []
#image_data = get_data_fits_wise(img_dir, img_redm_id, band, img_arcsec)
#image_data_zoom = zoom_image_data(image_data, img_pix)
#image_concat.append(image_data_zoom)
#check_image = apply_bright_objects_mask(check_image)
#image_data_zoom = apply_aperture_mask(image_data_zoom, img_mpc, 1.)
#plot_image_data(image_data_zoom, out_dir, band)
#final_image = np.zeros(shape=image_concat[0].shape)
#for image in image_concat:
#    final_image += image
#final_image = final_image/count
#plot_image_data(final_image, out_dir, band)

In [ ]:
img_pix